<a href="https://colab.research.google.com/github/Leila828/DeepLearning_projects/blob/master/Stop_Your_AI_From_Hallucinating.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install -q -U google.generativeai pydantic

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 90.6/90.6 kB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.1/155.1 kB 7.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 463.6/463.6 kB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 47.0 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
gradio 5.50.0 requires pydantic<=2.12.3,>=2.0, but you have pydantic 2.12.5 which is incompatible.


In [ ]:
# !pip install -q -U google-generativeai pydantic

import google.generativeai as genai
from pydantic import BaseModel
from typing import List
import json

# 1. Setup API Key
# Replace 'YOUR_API_KEY' or use colab userdata
from google.colab import userdata
GOOGLE_API_KEY = userdata.get('GOOGLE_API_KEY')
genai.configure(api_key=GOOGLE_API_KEY)


model_name = "models/gemini-2.5-flash"

# 2. Define the Schema
class Product(BaseModel):
    name: str
    quantity: int
    price: float

class OrderInfo(BaseModel):
    customer_name: str
    order_id: str
    items: List[Product]
    total_amount: float

# 3. Input Data
messy_email = """
From: Sarah Jones <sjones@email.com>
Subject: Need help with order #AX-9921
Hey team, I just realized I ordered 2 Coffee Mugs ($15 each) and 1 Espresso Machine ($299).
Can you make sure this ships fast?
"""

# --- STEP 4: GENERATE UNSTRUCTURED (THE "BAD" WAY) ---
model_text = genai.GenerativeModel(model_name=model_name)
prompt = f"Extract order info from this email: {messy_email}"
response_text = model_text.generate_content(prompt).text

# --- STEP 5: GENERATE STRUCTURED (THE "GOOD" WAY) ---
model_json = genai.GenerativeModel(
    model_name,
    generation_config={
        "response_mime_type": "application/json",
        "response_schema": OrderInfo
    }
)
response_json = model_json.generate_content(prompt).text

# --- STEP 6: THE COMPARISON PROOF ---
print("--- OUTPUT 1: UNSTRUCTURED TEXT ---")
print(response_text)
try:
    # Attempting to use the data in code
    tax = float(response_text) * 0.1  # This will CRASH
except Exception as e:
    print(f"\n❌ CODE ERROR: I cannot calculate tax because the output is just a bunch of sentences!")

print("\n" + "="*40 + "\n")

print("--- OUTPUT 2: STRUCTURED JSON ---")
data = json.loads(response_json)
print(response_json)

# Real-world logic using the data
tax = data['total_amount'] * 0.1
print(f"\n✅ CODE SUCCESS: The tax for {data['customer_name']} is ${tax:.2f}")
print(f"I can also loop through the {len(data['items'])} items easily.")

--- OUTPUT 1: UNSTRUCTURED TEXT ---
Here's the extracted order information:

*   **Order ID:** AX-9921

*   **Items:**
    *   Coffee Mugs: 2 @ $15 each = $30.00
    *   Espresso Machine: 1 @ $299 each = $299.00

*   **Total Order Value:** $329.00

❌ CODE ERROR: I cannot calculate tax because the output is just a bunch of sentences!


--- OUTPUT 2: STRUCTURED JSON ---
{"customer_name":"Sarah Jones","items":[{"name":"Coffee Mugs","price":15,"quantity":2},{"name":"Espresso Machine","price":299,"quantity":1}],"order_id":"AX-9921","total_amount":329}

✅ CODE SUCCESS: The tax for Sarah Jones is $32.90
I can also loop through the 2 items easily.
